In [5]:
%matplotlib inline
from numba import jit
import numpy as np
import time

import CyMod
import _SwigMod as SwigMod

import pyopencl as cl

In [6]:
@jit
def Num_NpDot(a, b):
    return np.dot(a, b)
@jit    
def Num_Dot(a, b):
    c = np.zeros((a.shape[0], b.shape[1]))
    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            for k in range(a.shape[1]):
                c[i][j] += a[i][k] * b[k][j]
    return c

In [7]:
plat    = cl.get_platforms()
devices = plat[0].get_devices()
devices

[<pyopencl.Device 'Intel(R) Core(TM) i3-3240 CPU @ 3.40GHz' on 'Intel(R) OpenCL' at 0x1e8947b9820>,
 <pyopencl.Device 'Intel(R) HD Graphics 2500' on 'Intel(R) OpenCL' at 0x1e892ad7f50>]

In [8]:
ctx     = cl.Context([devices[1]])
ctx.get_info(cl.context_info.DEVICES)

[<pyopencl.Device 'Intel(R) HD Graphics 2500' on 'Intel(R) OpenCL' at 0x1e892ad7f50>]

In [9]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

prg = cl.Program(ctx, """
    __kernel void OpenCL_Dot(const unsigned int n1, const unsigned int n2,
                            __global float * mat1, __global float * mat2,
                            __global float * res) {

        int i = get_global_id(0);
        int j = get_global_id(1);

        float tmp = 0;

        for (int k = 0; k < n1; k++){
            tmp += mat1[i*n1+k] * mat2[k*n2+j];
            res[i*n2+j] = tmp;
        }
        if(i==0 && j==0)res[i*n2+j] = 10.0;
            
    }
    """).build()

D:\Users\Chachay\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1476: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [10]:
N = 1000
# Generate NxN randomized matrix
ma = np.random.rand(N, N)
mb = np.random.rand(N, N)

In [11]:
print("1. Numpy Native\n   ", end='')
%timeit mc = np.dot(ma, mb)

1. Numpy Native
   10 loops, best of 3: 188 ms per loop


In [12]:
print("2. Namba Simple Mult\n   ", end='')
%timeit mc = Num_Dot(ma, mb)

print("3. Namba Numpy\n   ", end='')
%timeit mc = Num_NpDot(ma, mb)

2. Namba Simple Mult
   1 loop, best of 3: 8.11 s per loop
3. Namba Numpy
   The slowest run took 12.82 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 57.1 ms per loop


In [13]:
#print("4. Cython Simple Mult\n   ", end='')
#%timeit mc = CyMod.Cy_Dot(ma,mb)

print("5. Cython Numpy\n   ", end='')
%timeit mc = CyMod.Cy_NpDot(ma,mb)

5. Cython Numpy
   1 loop, best of 3: 174 ms per loop


In [14]:
print("6. Swig Simple Mult\n   ", end='')
%timeit mc = SwigMod.Swig_Dot(ma,mb)

# print("7. Swig MKL cblas_dgemm\n   ", end='')
# %timeit mc = SwigMod.Swig_Dot_MKL(ma,mb)

6. Swig Simple Mult
   1 loop, best of 3: 7.58 s per loop


※ OpenCLはメモリー転送のタイミング管理が悪いのか計算の結果が不正確。

In [15]:
def OpenCL_simpDot(a, b):
    global queue
    c = np.zeros((a.shape[0], b.shape[1]))
    
    mf = cl.mem_flags
    ma_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
    mb_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
    mc_buf = cl.Buffer(ctx, mf.WRITE_ONLY, c.nbytes)

    prg.OpenCL_Dot(queue, c.shape, None, 
                    np.int32(a.shape[1]), np.int32(b.shape[1]),
                    ma_buf, mb_buf, mc_buf)
    queue.finish()
    cl.enqueue_copy(queue, c, mc_buf)
    return c

print("8. OpenCL Simple Mult(Parallel)\n   ", end='')
%timeit mc = OpenCL_simpDot(ma,mb)

8. OpenCL Simple Mult(Parallel)
   1 loop, best of 3: 1.01 s per loop


# 参考文献
* [Python \(NumPy\) と Common Lisp \(LLA\) で行列積の実行速度を比較する \- 不確定特異点](http://tanakahx.hatenablog.com/entry/2015/09/25/070000)
* [Parallel programming with opencl and python – prototype](http://www.nehalemlabs.net/prototype/blog/2014/04/28/parallel-programming-with-opencl-and-python/)
* [10\.3: OpenCL](http://bathompso.com/education/compphys/parallel/opencl/)

* [Multiplying Matrices Using dgemm \| Intel® Software](https://software.intel.com/en-us/node/529735)
* [Intel® Math Kernel Library Link Line Advisor \| Intel® Software](https://software.intel.com/en-us/articles/intel-mkl-link-line-advisor)
* [cblas\_dgemm \- Accelerate \| Apple Developer Documentation](https://developer.apple.com/reference/accelerate/1513282-cblas_dgemm?language=objc)
